In [1]:
import pandas as pd
import sys
import os

import psycopg2

In [2]:
onlydirs = [f for f in os.listdir(os.getcwd()) if os.path.isdir(os.path.join(os.getcwd(), f)) and f[0] != '.']

In [3]:
onlydirs

['besoeg', 'besoeg2014-2019', 'besoegdokk1', 'besoegende']

In [4]:
os.chdir(os.getcwd()+'/'+'besoeg2014-2019')
os.getcwd()


'C:\\Users\\Jonas Friis\\Documents\\Universitet\\Bachelors-Thesis\\Implementation\\Data\\besoeg2014-2019'

In [5]:
param_dic = {
    "host"      : "localhost",
    "database"  : "bachelorBesoeg2014",
    "user"      : "postgres",
    "password"  : "password",
    "port"      : "5432"
}

In [6]:
file_in = 'besoeg2014-2019.csv'    
    
files = [f for f in os.listdir(os.getcwd()) if os.path.isfile(os.path.join(os.getcwd(), f)) and f != file_in]

In [7]:
files

['besoeg2014-2019775100.csv',
 'besoeg2014-2019775120.csv',
 'besoeg2014-2019775122.csv',
 'besoeg2014-2019775126.csv',
 'besoeg2014-2019775127.csv',
 'besoeg2014-2019775130.csv',
 'besoeg2014-2019775140.csv',
 'besoeg2014-2019775144.csv',
 'besoeg2014-2019775146.csv',
 'besoeg2014-2019775147.csv',
 'besoeg2014-2019775150.csv',
 'besoeg2014-2019775160.csv',
 'besoeg2014-2019775164.csv',
 'besoeg2014-2019775167.csv',
 'besoeg2014-2019775168.csv',
 'besoeg2014-2019775170.csv']

In [8]:
def execQuery(params_dic, query):
    try:
        connection = psycopg2.connect(**params_dic)
        cursor = connection.cursor()
        cursor.execute(query)
        record = cursor.fetchall()
        return record
    except (Exception, psycopg2.Error) as error :
        connection = False
        print ("Error while connecting to PostgreSQL", error)
    finally:
        if(connection):
            cursor.close()
            connection.close()
            print("Executed query and closed connection.")

#result = execQuery("""select content, type_id from article  ORDER BY article_id ASC LIMIT 200000 ;""")

In [9]:
def makeTable(params_dic, query):
    try:
        connection = psycopg2.connect(**params_dic)
        cursor = connection.cursor()
        cursor.execute(query)
    except (Exception, psycopg2.Error) as error :
        connection = False
        print ("Error while connecting to PostgreSQL", error)
    finally:
        if(connection):
            connection.commit()
            cursor.close()
            connection.close()
            print("Executed query and closed connection.")

In [10]:
for f in files:
    print(f[15:21])

775100
775120
775122
775126
775127
775130
775140
775144
775146
775147
775150
775160
775164
775167
775168
775170


In [11]:
for f in files:
    table = """
    CREATE TABLE {tablename} (
    time_ date,
    count_ int,
    PRIMARY KEY (time_)
    );
    """.format(tablename = '_'+f[15:21])
    #print(table)
    makeTable(param_dic,table)

Executed query and closed connection.
Executed query and closed connection.
Executed query and closed connection.
Executed query and closed connection.
Executed query and closed connection.
Executed query and closed connection.
Executed query and closed connection.
Executed query and closed connection.
Executed query and closed connection.
Executed query and closed connection.
Executed query and closed connection.
Executed query and closed connection.
Executed query and closed connection.
Executed query and closed connection.
Executed query and closed connection.
Executed query and closed connection.


In [12]:

import psycopg2.extras as extras

In [13]:
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1)
    print("Connection successful")
    return conn
def execute_values(conn, df, table):
    """
    Using psycopg2.extras.execute_values() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()

In [14]:
conn = connect(param_dic)
for f in files:
    df = pd.read_csv(f,sep=',') 
    df = df.drop(columns=['libno'])
    df.columns = ['time_', 'count_']
    execute_values(conn, df, '_'+f[15:21])

conn.close()
    

Connecting to the PostgreSQL database...
Connection successful
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
